# Clause Segmentation

Segment contract text into clauses using rule-based heuristics + sentence transformers clustering.

In [ ]:
%python
%pip install transformers torch sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")

def segment_clauses(text):
    parts = [p.strip() for p in text.split("\n") if len(p.strip()) > 20]
    return parts

def embed_clauses(parts):
    return model.encode(parts, convert_to_numpy=True).tolist()
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf

segment_udf = udf(segment_clauses, ArrayType(StringType()))
silver_path = "abfss://silver@ragstorage4122025.dfs.core.windows.net"
silver_df = (
    spark.read
    .format("delta")
    .load(silver_path)
)

gold_df = silver_df.withColumn("clauses", segment_udf(col("text_clean")))
gold_path = "abfss://gold@ragstorage4122025.dfs.core.windows.net"

gold_df.write.format("delta").mode("overwrite").save(gold_path)
